In [ ]:
import cv2
import pytesseract
import pandas as pd
from PIL import Image
import numpy as np
import os
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\charbel.nasr\Tesseract-OCR-New\tesseract.exe'
directory = "Images"

In [ ]:
def list_data(boxes,image):
    lst_data=[]
    for x,b in enumerate(boxes.splitlines()):
        if x!=0:
            b=b.split()
            if len(b)==12:
                lst_data.append((int(b[6]),int(b[7]),int(b[8]),int(b[9]),b[11]))
                # get the x,y,width,height
                x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                cv2.rectangle(image,(x,y),(w+x,h+y),(255,0,0),1)

    return lst_data 

In [ ]:
def get_entity(lst_data):
    direction_top = lst_data[0][1]
    previous_top = direction_top+lst_data[0][3]
    direction = lst_data.pop(0)[4]
    entité = direction
    current_top = lst_data[0][1]
    return direction,entité,current_top,previous_top

In [ ]:
def get_responsable_status(current_top, previous_top, lst_data):
    status = '' 
    top = 0
    while  lst_data[0][4] in ('='):
        lst_data.pop(0)
    while (abs(current_top-previous_top) <= 70 and lst_data[0][4] not in ['Responsable', 'Secrétaire', 'Personnes','Entités'] ):
        status += lst_data[0][4]+' '
        top = lst_data[0][1]
        previous_top = lst_data[0][1]+lst_data[0][3]
        lst_data.pop(0)
        current_top = lst_data[0][1] if lst_data else 0
    return status,previous_top

In [ ]:
def get_responsable_name(previous_top, lst_data):
    nom ,prénom, top= '','',''
    current_top = lst_data[0][1]
    while (abs(current_top- previous_top) <= 85 and lst_data[0][4] not in ['Secrétaire','Secrétaire:', 'Personnes','Entités'] ):
        text = lst_data[0][4]
        if text not in ('Responsable',':'):
            if  prénom == '':
                prénom = text
            else:
                nom+=text+' '
        previous_top = lst_data[0][1]+lst_data[0][3]
        lst_data.pop(0)
        current_top = lst_data[0][1] if lst_data else 0
    return prénom,nom,previous_top

In [ ]:
def get_secrétaire_data(previous_top, lst_data):
    prénom,nom ='',''
    current_top = lst_data[0][1]
    while (abs(current_top- previous_top )<= 110 and lst_data[0][4] not in ['Personnes','Entités']):
        text = lst_data[0][4]
        if text not in ('Secrétaire',':','Secrétaire:'):
            if  prénom == '':
                prénom = text
            else:
                nom+=text+' '
        previous_top = lst_data[0][1]+lst_data[0][3]
        lst_data.pop(0)
        current_top=lst_data[0][1] if lst_data else 0
    return prénom,nom,previous_top

In [ ]:
def get_personnes_rattachées(previous_top, df, lst_data, dict):
    prénom,nom= '',''
    current_top = lst_data[0][1]
    # Remove Personnes rattachées Text
    while  lst_data[0][4] in ('Personnes', 'rattachées','*','rattachees'):
        previous_top = lst_data[0][1] + lst_data[0][3]
        lst_data.pop(0)
        current_top = lst_data[0][1]
    
    while (abs(current_top - previous_top)  <= 85 and lst_data[0][4] not in ['Entités']):
        top = current_top
        # check if we are at same line
        while current_top in range(top-11,top+11): 
            text = lst_data[0][4]
            if  prénom == '':
                prénom = text
            else:
                nom+=text+' '
            top = lst_data[0][1]
            lst_data.pop(0)
            current_top=lst_data[0][1] if lst_data else 0
        dict['Prénom'],dict['Nom'] = prénom,nom
        dict['Email'] = dict['Prénom']+'.'+dict['Nom']+'@cnp.fr'
        dict['Status'] = 'Personnes rattachées'
        df = df.append(dict,ignore_index=True)
        previous_top = top
        prénom,nom= '',''
    return df, previous_top

In [ ]:
def get_names(current_top, lst_data):
    prénom,nom= '',''
    # get names that are on same line with entity name
    for x in [t for t in lst_data if ((t[1] in range(current_top-11,current_top+25)) and(t[4] != '=' ) and (t[0] not in range(0,85)))]:  
        if  prénom == '':
            prénom = x[4]
        else:
            nom+=x[4]+' '
    return prénom,nom

In [ ]:
def get_entités_rattachées(previous_top , df, lst_data , dict):
    current_top = lst_data[0][1]
    # Remove Entité rattachées Text
    while  lst_data[0][4] in ('Entités', 'rattachées'):
        previous_top = lst_data[0][1] + lst_data[0][3]
        lst_data.pop(0)
        current_top=lst_data[0][1]
    for rec in [t for t in lst_data if ((t[0] in range(0,88)))]:
        dict['Entité']=rec[4]
        dict['Prénom'],dict['Nom'] = get_names(rec[1], lst_data)
        dict['Email'] =  dict['Prénom']+'.'+dict['Nom']+'@cnp.fr'
        dict['Status'] = 'Entité rattachées'
        df = df.append(dict,ignore_index=True)
    return df

In [ ]:
def get_image(filepath):
    img = Image.open(filepath)
    # convert to grayscale
    img = img.convert("L")
    norm_img = np.zeros((img.height, img.width))
    img = Image.fromarray(cv2.normalize(np.array(img), norm_img, 0, 255, cv2.NORM_MINMAX))
    return img

In [ ]:
df = pd.DataFrame(columns=['Nom', 'Prénom', 'Entité', 'Direction','Status','Email','pid','visible'])
directory = "Images"
for folder_name in os.listdir(directory):
    if os.path.isdir(os.path.join(directory, folder_name)):
        print("Processing folder:", folder_name)
    for filename in os.listdir(os.path.join(directory, folder_name)):
        filename = filename.replace(".PNG", ".png")
        filepath = os.path.join(os.path.join(directory, folder_name), filename)
        img = get_image(filepath)
        # Convert Image from Pillow To OpenCV
        numpy_array = np.asarray(img)
        boxes = pytesseract.image_to_data(numpy_array, lang='eng+fra', config='--psm 6')
        lst_data = list_data(boxes, numpy_array)
        dict ={}
        dict['Direction'],dict['Entité'],current_top,previous_top = get_entity(lst_data)
        dict['Status'], previous_top= get_responsable_status(current_top, previous_top, lst_data)
        if lst_data and 'Responsable' in lst_data[0][4]:
            # getting name of responsable
            dict['Prénom'],dict['Nom'],previous_top = get_responsable_name(previous_top, lst_data)
            dict['Email'] =  dict['Prénom']+'.'+dict['Nom']+'@cnp.fr'
            # add responsable data
            df = df.append(dict,ignore_index=True)

        if  lst_data and 'Secrétaire' in lst_data[0][4]: 
            dict['Prénom'],dict['Nom'],previous_top = get_secrétaire_data(previous_top, lst_data)
            dict['Email'] =  dict['Prénom']+'.'+dict['Nom']+'@cnp.fr'
            dict['Status'] = 'Secrétaire'
            df = df.append(dict,ignore_index=True)

        if lst_data and 'Personnes'in lst_data[0][4]:
            df,previous_top = get_personnes_rattachées(previous_top, df ,lst_data, dict)   

        if lst_data and 'Entités'in lst_data[0][4]:
            df = get_entités_rattachées(previous_top, df, lst_data ,dict)
        df = pd.concat([df], ignore_index=True)

for  index, row in df.iterrows():
    locate_row = df.loc[(df['Entité'] == df['Direction']) & 
       (df['Entité'] == row['Direction']) & 
       (~df['Status'].isin(['Secrétaire', 'Personnes rattachées', 'Entité rattachées']))]
    if (not locate_row.empty) and (row['Status'] in ['Secrétaire', 'Personnes rattachées', 'Entité rattachées']):
        indice = locate_row.index[0]
    else:
        indice = ''
    df.at[index, 'pid'] = indice

for  index, row in df.loc[~df['Status'].isin(['Secrétaire', 'Personnes rattachées', 'Entité rattachées'])].iterrows():
    locate_entity = df.loc[(df['Prénom'] == row['Prénom']) & 
       (df['Nom'] == row['Nom']) & 
       (df['Status'] == 'Entité rattachées')&
       (df['Entité'] == row['Entité'])&
       (df['Entité'] == row['Direction'])]
    if not locate_entity.empty:
        df.at[index, 'pid'] = locate_entity['pid'].values[0]

df['visible'] = True
df.loc[df['Status'].isin(['Secrétaire', 'Personnes rattachées','Entité rattachées']), 'visible'] = False
df.to_csv('Excel/Latest1BE.csv', index=True, header=True, encoding='utf-8-sig',index_label='id')